In [117]:
from scripts.fe import FeatureEngineering
from scripts.fs import FeatureSelection
from scripts.fs import train_test_split_function
from scripts.cross_val import plot_results
from scripts.cross_val import find_best_score
from scripts.cross_val import plot_results
from sklearn.ensemble import RandomForestClassifier
from scripts.modelling import prediction_to_csv
from sklearn.metrics import mean_absolute_error


In [118]:
from scripts.lazy_model import lazy_model

In [119]:
df = FeatureEngineering()

In [120]:
df.columns

Index(['building_id_x', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use

In [121]:
selected_features = ['age','damage_grade']
selected_cdf = FeatureSelection(df, selected_features)

In [131]:
selected_cdf

,age,damage_grade
0,30,3
1,10,2
2,10,3
3,10,2
4,30,3
...,...,...
260596,55,2
260597,0,3
260598,55,3
260599,10,2


In [134]:
y_test

147315    2
39728     3
255109    3
190884    2
133178    1
         ..
4818      3
63697     2
164994    3
70445     2
177474    2
Name: damage_grade, Length: 52121, dtype: int64

In [139]:
X_train, X_test, y_train, y_test = train_test_split_function(selected_cdf, 'damage_grade')

In [140]:
from scripts.lazy_model import lazy_model

In [141]:
fitted_model,mae = lazy_model(X_train, X_test, y_train, y_test,RandomForestClassifier)

AttributeError: 'DataFrame' object has no attribute '_validate_params'

In [112]:
mae

0.4299418660424781

In [113]:
def prediction_to_csv(model):

    X_test = pd.read_csv('data/test_values.csv')
    X_test = pd.DataFrame(X_test.age)
    
   # Assuming you have already trained your model and made predictions
    y_pred = model.predict(X_test)
    np.savetxt('data/predictions.csv', y_pred, delimiter=',', fmt='%d')
    # Set index=False to avoid writing row numbers as a column in the csv file
    

In [114]:
X_test.head()

,age
195839,15
28925,5
243800,15
48064,25
235855,10


In [115]:
import pandas as pd
import numpy as np
final_csv = prediction_to_csv(fitted_model)


In [116]:
final_csv